In [1]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [12]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHAIGPRX1164_final.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:53:58:58:42:16:27.59%:4.6015E-6:59:53:36:6...
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:42:62:62:49:13:20.97%:6.0557E-5:59:57:38:1...
2,chr1,65649,rs1166163776,G,A,.,PASS,ADP=28;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1166163776...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:26:28:28:20:8:28.57%:2.188E-3:51:50:10:10:4:4
3,chr1,69428,rs140739101,T,G,.,PASS,"ADP=63;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.981,0.0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:64:63:63:44:19:30.16%:3.8563E-7:47:55:17:2...
4,chr1,450718,rs2808347,T,C,.,PASS,ADP=80;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=OR4F...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:51:80:80:64:16:20%:6.64E-6:48:53:59:5:11:5
...,...,...,...,...,...,...,...,...,...,...
40388,chrX,155941873,rs5940625,G,C,.,PASS,"ADP=24;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6446,.,...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:135:24:24:0:24:100%:3.101E-14:0:56:0:0:21:3
40389,chrY,13986392,.,T,C,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;CSQ=C|missense_va...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:52:10:10:0:10:100%:5.4125E-6:0:58:0:0:6:4
40390,chrY,14840455,rs753595702,T,C,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:52:10:10:0:10:100%:5.4125E-6:0:38:0:0:10:0
40391,chrY,14840785,rs767683335,C,T,.,PASS,ADP=47;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:47:47:0:47:100%:6.1512E-28:0:36:0:0:0:47


In [13]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHAIGPRX1164_final.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)
vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR']]
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4
2,chr1,65649,rs1166163776,G,A,.,PASS,ADP=28;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1166163776...,0/1,26,28,28,20,8,28.57%,2.188E-3,10,10,4,4
3,chr1,69428,rs140739101,T,G,.,PASS,"ADP=63;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.981,0.0...",0/1,64,63,63,44,19,30.16%,3.8563E-7,17,27,11,8
4,chr1,450718,rs2808347,T,C,.,PASS,ADP=80;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=OR4F...,0/1,51,80,80,64,16,20%,6.64E-6,59,5,11,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40388,chrX,155941873,rs5940625,G,C,.,PASS,"ADP=24;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6446,.,...",1/1,135,24,24,0,24,100%,3.101E-14,0,0,21,3
40389,chrY,13986392,.,T,C,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;CSQ=C|missense_va...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,6,4
40390,chrY,14840455,rs753595702,T,C,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,10,0
40391,chrY,14840785,rs767683335,C,T,.,PASS,ADP=47;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,1/1,255,47,47,0,47,100%,6.1512E-28,0,0,0,47


In [14]:
vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf['GT'] = vcf['GT'].astype(str)
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous
2,chr1,65649,rs1166163776,G,A,.,PASS,ADP=28;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1166163776...,0/1,26,28,28,20,8,28.57%,2.188E-3,10,10,4,4,1,0,Heterozygous
3,chr1,69428,rs140739101,T,G,.,PASS,"ADP=63;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.981,0.0...",0/1,64,63,63,44,19,30.16%,3.8563E-7,17,27,11,8,1,0,Heterozygous
4,chr1,450718,rs2808347,T,C,.,PASS,ADP=80;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=OR4F...,0/1,51,80,80,64,16,20%,6.64E-6,59,5,11,5,1,0,Heterozygous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40388,chrX,155941873,rs5940625,G,C,.,PASS,"ADP=24;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6446,.,...",1/1,135,24,24,0,24,100%,3.101E-14,0,0,21,3,0,1,Homozygous
40389,chrY,13986392,.,T,C,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;CSQ=C|missense_va...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,6,4,0,1,Homozygous
40390,chrY,14840455,rs753595702,T,C,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,10,0,0,1,Homozygous
40391,chrY,14840785,rs767683335,C,T,.,PASS,ADP=47;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,1/1,255,47,47,0,47,100%,6.1512E-28,0,0,0,47,0,1,Homozygous


In [15]:
vcf["Gene_Name"] = vcf["INFO"].str.extract('GENEINFO=(?P<GENEINFO>.+?);')
#vcf['Gene Name'] = vcf['Gene_Name'].apply(lambda x: ','.join([segment.split(':')[0] for segment in x.split('|')]) if pd.notnull(x) else '')
vcf['Gene Name'] = vcf['Gene_Name'].apply(lambda x: ','.join(set([segment.split(':')[0] for segment in x.split('|')])) if pd.notnull(x) else '')
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,
2,chr1,65649,rs1166163776,G,A,.,PASS,ADP=28;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1166163776...,0/1,26,28,28,20,8,28.57%,2.188E-3,10,10,4,4,1,0,Heterozygous,NaN,
3,chr1,69428,rs140739101,T,G,.,PASS,"ADP=63;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.981,0.0...",0/1,64,63,63,44,19,30.16%,3.8563E-7,17,27,11,8,1,0,Heterozygous,OR4F5:79501,OR4F5
4,chr1,450718,rs2808347,T,C,.,PASS,ADP=80;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=OR4F...,0/1,51,80,80,64,16,20%,6.64E-6,59,5,11,5,1,0,Heterozygous,OR4F29:729759,OR4F29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40388,chrX,155941873,rs5940625,G,C,.,PASS,"ADP=24;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6446,.,...",1/1,135,24,24,0,24,100%,3.101E-14,0,0,21,3,0,1,Homozygous,VAMP7:6845,VAMP7
40389,chrY,13986392,.,T,C,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;CSQ=C|missense_va...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,6,4,0,1,Homozygous,NaN,
40390,chrY,14840455,rs753595702,T,C,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,10,0,0,1,Homozygous,NLGN4Y:22829,NLGN4Y
40391,chrY,14840785,rs767683335,C,T,.,PASS,ADP=47;WT=0;HET=0;HOM=1;NC=0;ASP;G5;G5A;GENEIN...,1/1,255,47,47,0,47,100%,6.1512E-28,0,0,0,47,0,1,Homozygous,NLGN4Y:22829,NLGN4Y


In [16]:
vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['csq'] = vcf['CSQ'].str.split(',')
vcf = vcf.explode('csq')
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name,CSQ,csq
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|5_prime_UTR_variant|MODIFIER|OR4F5|ENSG00000...
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER||ENSG000002...
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|intron_variant|MODIFIER|OR4F5|ENSG0000018609...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|non_coding_transcript_exon_variant|MODIFIER|...
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...


In [17]:
########################################################### Required columns extraction from the CSQ column ####################
vcf['ClinVar_CLNDN'] = vcf['csq'].str.split('|').str[82]
vcf['CLIN_SIG'] = vcf['csq'].str.split('|').str[70]
vcf['ClinVar_CLNREVSTAT'] = vcf['csq'].str.split('|').str[81]
vcf['ClinVar'] = vcf['csq'].str.split('|').str[79]
vcf['HGVSC'] = vcf['csq'].str.split('|').str[10]
vcf['HGVSP'] = vcf['csq'].str.split('|').str[11]
vcf['PolyPhen'] = vcf['csq'].str.split('|').str[38]
vcf['BIOTYPE'] = vcf['csq'].str.split('|').str[7]
vcf['EXON'] = vcf['csq'].str.split('|').str[8]
vcf['INTRON'] = vcf['csq'].str.split('|').str[9]
vcf['Protein_position'] = vcf['csq'].str.split('|').str[14]
vcf['Amino_acids'] = vcf['csq'].str.split('|').str[15]
vcf['Codons'] = vcf['csq'].str.split('|').str[16]
vcf['STRAND'] = vcf['csq'].str.split('|').str[19]
vcf['PUBMED'] = vcf['csq'].str.split('|').str[73]
vcf['Consequence'] = vcf['csq'].str.split('|').str[1]
vcf['IMPACT'] = vcf['csq'].str.split('|').str[2]
vcf['SIFT'] = vcf['csq'].str.split('|').str[37]
################################################## Frequency columns extraction ################################################
vcf['AF'] = vcf['csq'].str.split('|').str[42]
vcf['AFR_AF'] = vcf['csq'].str.split('|').str[43]
vcf['AMR_AF'] = vcf['csq'].str.split('|').str[44]
vcf['EAS_AF'] = vcf['csq'].str.split('|').str[45]
vcf['EUR_AF'] = vcf['csq'].str.split('|').str[46]
vcf['SAS_AF'] = vcf['csq'].str.split('|').str[47]
vcf['gnomADe_AF'] = vcf['csq'].str.split('|').str[48]
vcf['gnomADe_AFR_AF'] = vcf['csq'].str.split('|').str[49]
vcf['gnomADe_AMR_AF'] = vcf['csq'].str.split('|').str[50]
vcf['gnomADe_ASJ_AF'] = vcf['csq'].str.split('|').str[51]
vcf['gnomADe_EAS_AF'] = vcf['csq'].str.split('|').str[52]
vcf['gnomADe_FIN_AF'] = vcf['csq'].str.split('|').str[53]
vcf['gnomADe_NFE_AF'] = vcf['csq'].str.split('|').str[54]
vcf['gnomADe_OTH_AF'] = vcf['csq'].str.split('|').str[55]
vcf['gnomADe_SAS_AF'] = vcf['csq'].str.split('|').str[56]
vcf['gnomADg_AF'] = vcf['csq'].str.split('|').str[57]
vcf['gnomADg_AFR_AF'] = vcf['csq'].str.split('|').str[58]
vcf['gnomADg_AMI_AF'] = vcf['csq'].str.split('|').str[59]
vcf['gnomADg_AMR_AF'] = vcf['csq'].str.split('|').str[60]
vcf['gnomADg_ASJ_AF'] = vcf['csq'].str.split('|').str[61]
vcf['gnomADg_EAS_AF'] = vcf['csq'].str.split('|').str[62]
vcf['gnomADg_FIN_AF'] = vcf['csq'].str.split('|').str[63]
vcf['gnomADg_MID_AF'] = vcf['csq'].str.split('|').str[64]
vcf['gnomADg_NFE_AF'] = vcf['csq'].str.split('|').str[65]
vcf['gnomADg_OTH_AF'] = vcf['csq'].str.split('|').str[66]
vcf['gnomADg_SAS_AF'] = vcf['csq'].str.split('|').str[67]
vcf['MAX_AF'] = vcf['csq'].str.split('|').str[68]
vcf['MAX_AF_POPS'] = vcf['csq'].str.split('|').str[69]
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name,CSQ,csq,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSC,HGVSP,PolyPhen,BIOTYPE,EXON,INTRON,Protein_position,Amino_acids,Codons,STRAND,PUBMED,Consequence,IMPACT,SIFT,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed_unprocessed_pseudogene,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|5_prime_UTR_variant|MODIFIER|OR4F5|ENSG00000...,,,,,ENST00000641515.2:c.-36C>T,,,protein_coding,2/3,,,,,1,,5_prime_UTR_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER||ENSG000002...,,,,,,,,lncRNA,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed_unprocessed_pseudogene,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg_AMI
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|intron_variant|MODIFIER|OR4F5|ENSG0000018609...,,,,,ENST00000641515.2:c.9+18C>T,,,protein_coding,,2/2,,,,1,,intron_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg_AMI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382868.5:c.2247C>T,ENSP00000372320.1:p.Ala749%3D,,protein_coding,8/8,,749,A,gcC/gcT,1,,synonymous_variant,LOW,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382872.5:c.1572C>T,ENSP00000372325.1:p.Ala524%3D,,protein_coding,6/6,,524,A,gcC/gcT,1,,synonymous_variant,LOW,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEIN

In [18]:
vcf['Protein Position and Amino Acid'] = vcf['Amino_acids'].str[0] + vcf['Protein_position'] + np.where(vcf['Amino_acids'].str[-1] == vcf['Amino_acids'].str[0], '', vcf['Amino_acids'].str[-1])
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name,CSQ,csq,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSC,HGVSP,PolyPhen,BIOTYPE,EXON,INTRON,Protein_position,Amino_acids,Codons,STRAND,PUBMED,Consequence,IMPACT,SIFT,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,Protein Position and Amino Acid
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed_unprocessed_pseudogene,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ,NaN
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|5_prime_UTR_variant|MODIFIER|OR4F5|ENSG00000...,,,,,ENST00000641515.2:c.-36C>T,,,protein_coding,2/3,,,,,1,,5_prime_UTR_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ,NaN
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER||ENSG000002...,,,,,,,,lncRNA,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ,NaN
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed_unprocessed_pseudogene,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg_AMI,NaN
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|intron_variant|MODIFIER|OR4F5|ENSG0000018609...,,,,,ENST00000641515.2:c.9+18C>T,,,protein_coding,,2/2,,,,1,,intron_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg_AMI,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382868.5:c.2247C>T,ENSP00000372320.1:p.Ala749%3D,,protein_coding,8/8,,749,A,gcC/gcT,1,,synonymous_variant,LOW,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,A749
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382872.5:c.1572C>T,ENSP00000372325.1:p.Ala524%3D,,protein_coding,6/6,,524,A,gcC/gcT,1,,synonymous_variant,LOW,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,A524
40392,chrY,14840

In [19]:
vcf[['HGVSc', 'HGVSc (Transcript)']] = vcf['HGVSC'].str.split(':', 1, expand=True)
vcf[['HGVSp', 'HGVSp (Transcript)']] = vcf['HGVSP'].str.split(':', 1, expand=True)
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name,CSQ,csq,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSC,HGVSP,PolyPhen,BIOTYPE,EXON,INTRON,Protein_position,Amino_acids,Codons,STRAND,PUBMED,Consequence,IMPACT,SIFT,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,Protein Position and Amino Acid,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript)
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed_unprocessed_pseudogene,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ,NaN,,None,,None
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|5_prime_UTR_variant|MODIFIER|OR4F5|ENSG00000...,,,,,ENST00000641515.2:c.-36C>T,,,protein_coding,2/3,,,,,1,,5_prime_UTR_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ,NaN,ENST00000641515.2,c.-36C>T,,None
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER||ENSG000002...,,,,,,,,lncRNA,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ,NaN,,None,,None
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed_unprocessed_pseudogene,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg_AMI,NaN,,None,,None
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|intron_variant|MODIFIER|OR4F5|ENSG0000018609...,,,,,ENST00000641515.2:c.9+18C>T,,,protein_coding,,2/2,,,,1,,intron_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg_AMI,NaN,ENST00000641515.2,c.9+18C>T,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382868.5:c.2247C>T,ENSP00000372320.1:p.Ala749%3D,,protein_coding,8/8,,749,A,gcC/gcT,1,,synonymous_variant,LOW,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,A749,ENST00000382868.5,c.2247C>T,ENSP00000372320.1,p.Ala749%3D
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524

In [20]:
vcf_final = vcf.copy()
# Define the terms to remove
remove_terms = set(["not_specified", "not_provided"])

# Apply the filtering operation to 'Column1' only
vcf_final['ClinVar_CLNDN'] = vcf_final['ClinVar_CLNDN'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)


vcf_final['CLIN_SIG'] = vcf_final['CLIN_SIG'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)


vcf_final['ClinVar_CLNREVSTAT'] = vcf_final['ClinVar_CLNREVSTAT'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)

# Print the modified DataFrame
vcf_final

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name,CSQ,csq,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSC,HGVSP,PolyPhen,BIOTYPE,EXON,INTRON,Protein_position,Amino_acids,Codons,STRAND,PUBMED,Consequence,IMPACT,SIFT,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,Protein Position and Amino Acid,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript)
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed_unprocessed_pseudogene,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ,NaN,,None,,None
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|5_prime_UTR_variant|MODIFIER|OR4F5|ENSG00000...,,,,,ENST00000641515.2:c.-36C>T,,,protein_coding,2/3,,,,,1,,5_prime_UTR_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ,NaN,ENST00000641515.2,c.-36C>T,,None
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER||ENSG000002...,,,,,,,,lncRNA,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg_ASJ,NaN,,None,,None
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed_unprocessed_pseudogene,,,,,,1,,downstream_gene_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg_AMI,NaN,,None,,None
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,NaN,,T|downstream_gene_variant|MODIFIER|OR4G11P|ENS...,T|intron_variant|MODIFIER|OR4F5|ENSG0000018609...,,,,,ENST00000641515.2:c.9+18C>T,,,protein_coding,,2/2,,,,1,,intron_variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg_AMI,NaN,ENST00000641515.2,c.9+18C>T,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382868.5:c.2247C>T,ENSP00000372320.1:p.Ala749%3D,,protein_coding,8/8,,749,A,gcC/gcT,1,,synonymous_variant,LOW,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,A749,ENST00000382868.5,c.2247C>T,ENSP00000372320.1,p.Ala749%3D
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous_variant|LOW|NLGN4Y|ENSG0000016524

In [21]:
vcf_final = vcf_final.astype(str).applymap(lambda x: x.replace('&', ',').replace('_', ' '))
vcf_final

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name,CSQ,csq,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSC,HGVSP,PolyPhen,BIOTYPE,EXON,INTRON,Protein_position,Amino_acids,Codons,STRAND,PUBMED,Consequence,IMPACT,SIFT,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,Protein Position and Amino Acid,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript)
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed unprocessed pseudogene,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,,None,,None
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|5 prime UTR variant|MODIFIER|OR4F5|ENSG00000...,,,,,ENST00000641515.2:c.-36C>T,,,protein coding,2/3,,,,,1,,5 prime UTR variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,ENST00000641515.2,c.-36C>T,,None
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER||ENSG000002...,,,,,,,,lncRNA,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,,None,,None
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed unprocessed pseudogene,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,nan,,None,,None
1,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|intron variant|MODIFIER|OR4F5|ENSG0000018609...,,,,,ENST00000641515.2:c.9+18C>T,,,protein coding,,2/2,,,,1,,intron variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,nan,ENST00000641515.2,c.9+18C>T,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382868.5:c.2247C>T,ENSP00000372320.1:p.Ala749%3D,,protein coding,8/8,,749,A,gcC/gcT,1,,synonymous variant,LOW,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,A749,ENST00000382868.5,c.2247C>T,ENSP00000372320.1,p.Ala749%3D
40392,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous variant|LOW|NLGN4Y|ENSG0000016524

In [22]:
vcf_final['consequence'] = vcf_final['Consequence'].str.split(',').str[0]

df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/consequence.xlsx')

merged_1 = pd.merge(vcf_final, df_1, on='consequence', how='left', sort=False)

df_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/IMPACT.xlsx')

merged_2 = pd.merge(merged_1, df_2, on = 'IMPACT', how='left', sort=False)
merged_2

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name,CSQ,csq,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSC,HGVSP,PolyPhen,BIOTYPE,EXON,INTRON,Protein_position,Amino_acids,Codons,STRAND,PUBMED,Consequence,IMPACT,SIFT,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,Protein Position and Amino Acid,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),consequence,Consequence_score,IMPACT_score
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed unprocessed pseudogene,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,,None,,None,downstream gene variant,2/10,1.5
1,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|5 prime UTR variant|MODIFIER|OR4F5|ENSG00000...,,,,,ENST00000641515.2:c.-36C>T,,,protein coding,2/3,,,,,1,,5 prime UTR variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,ENST00000641515.2,c.-36C>T,,None,5 prime UTR variant,3/10,1.5
2,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER||ENSG000002...,,,,,,,,lncRNA,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,,None,,None,downstream gene variant,2/10,1.5
3,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed unprocessed pseudogene,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,nan,,None,,None,downstream gene variant,2/10,1.5
4,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|intron variant|MODIFIER|OR4F5|ENSG0000018609...,,,,,ENST00000641515.2:c.9+18C>T,,,protein coding,,2/2,,,,1,,intron variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,nan,ENST00000641515.2,c.9+18C>T,,None,intron variant,2/10,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351857,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382868.5:c.2247C>T,ENSP00000372320.1:p.Ala749%3D,,protein coding,8/8,,749,A,gcC/gcT,1,,synonymous variant,LOW,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,A749,ENST00000382868.5,c.2247C>T,ENSP00000372320.1,p.Ala749%3D,synonymo

In [23]:
df_gene = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/AIGPRX1164_genes.xlsx')

merged_2['Gene Match'] = 'No'
merged_2['Matched_Gene'] = ''
    
# Iterate through each gene in vcf['Gene']
for index, genes in merged_2['Gene Name'].iteritems():
    if isinstance(genes, str):
        gene_list = genes.split(',')
        for gene in gene_list:
            if gene in df_gene['Gene Name'].values:
                merged_2.at[index, 'Gene Match'] = 'Yes'
                merged_2.at[index, 'Matched_Gene'] = gene
                break
    
df_gene = df_gene.rename({'Gene Name':'Matched_Gene', 'Gene_Match':'Gene Match'}, axis=1)

merged_2 = merged_2.drop(columns=['Gene Match'], axis=1)

merged_2 = pd.merge(merged_2, df_gene, on= 'Matched_Gene', how = 'left', sort = False)
merged_2['Gene Match'] = merged_2['Gene Match'].fillna('No')
merged_2

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name,CSQ,csq,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSC,HGVSP,PolyPhen,BIOTYPE,EXON,INTRON,Protein_position,Amino_acids,Codons,STRAND,PUBMED,Consequence,IMPACT,SIFT,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,Protein Position and Amino Acid,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),consequence,Consequence_score,IMPACT_score,Matched_Gene,Gene Match
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed unprocessed pseudogene,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,,None,,None,downstream gene variant,2/10,1.5,,No
1,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|5 prime UTR variant|MODIFIER|OR4F5|ENSG00000...,,,,,ENST00000641515.2:c.-36C>T,,,protein coding,2/3,,,,,1,,5 prime UTR variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,ENST00000641515.2,c.-36C>T,,None,5 prime UTR variant,3/10,1.5,,No
2,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER||ENSG000002...,,,,,,,,lncRNA,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,,None,,None,downstream gene variant,2/10,1.5,,No
3,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed unprocessed pseudogene,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,nan,,None,,None,downstream gene variant,2/10,1.5,,No
4,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|intron variant|MODIFIER|OR4F5|ENSG0000018609...,,,,,ENST00000641515.2:c.9+18C>T,,,protein coding,,2/2,,,,1,,intron variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,nan,ENST00000641515.2,c.9+18C>T,,None,intron variant,2/10,1.5,,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351857,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382868.5:c.2247C>T,ENSP00000372320.1:p.Ala749%3D,,protein coding,8/8,,749,A,gcC/gcT,1,,synonymous variant,LOW,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,A749,ENST0000038286

In [24]:
merged_2['Gene Match'].value_counts()

No                            349398
Ovarian_health                  1456
PTE/VTE                          701
PCOS                             216
PTE/VTE_and_Ovarian_health        48
Ovarian_health_and_PCOS           43
Name: Gene Match, dtype: int64

In [27]:
df_3 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/AIGPRX1164_protein_Change.xlsx')
df_3

,Matched_Gene,Protein Position and Amino Acid,Alterations
0,F5,G1691A,Yes
1,F6,C677T,Yes
2,PGM,G20210A,Yes


In [29]:
merged_3 = pd.merge(merged_2, df_3, on=['Matched_Gene', 'Protein Position and Amino Acid'], how='left', sort=False)
merged_3['Alterations'] = merged_3['Alterations'].fillna('No')
merged_3

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,Gene_Name,Gene Name,CSQ,csq,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSC,HGVSP,PolyPhen,BIOTYPE,EXON,INTRON,Protein_position,Amino_acids,Codons,STRAND,PUBMED,Consequence,IMPACT,SIFT,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,Protein Position and Amino Acid,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),consequence,Consequence_score,IMPACT_score,Matched_Gene,Gene Match,Alterations
0,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed unprocessed pseudogene,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,,None,,None,downstream gene variant,2/10,1.5,,No,No
1,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|5 prime UTR variant|MODIFIER|OR4F5|ENSG00000...,,,,,ENST00000641515.2:c.-36C>T,,,protein coding,2/3,,,,,1,,5 prime UTR variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,ENST00000641515.2,c.-36C>T,,None,5 prime UTR variant,3/10,1.5,,No,No
2,chr1,65529,rs1473003496,C,T,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1473003496...,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER||ENSG000002...,,,,,,,,lncRNA,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,nan,,None,,None,downstream gene variant,2/10,1.5,,No,No
3,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,,,,,,,,transcribed unprocessed pseudogene,,,,,,1,,downstream gene variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,nan,,None,,None,downstream gene variant,2/10,1.5,,No,No
4,chr1,65591,rs1416222198,C,T,.,PASS,ADP=62;WT=0;HET=1;HOM=0;NC=0;ASP;RS=1416222198...,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,1,0,Heterozygous,nan,,T|downstream gene variant|MODIFIER|OR4G11P|ENS...,T|intron variant|MODIFIER|OR4F5|ENSG0000018609...,,,,,ENST00000641515.2:c.9+18C>T,,,protein coding,,2/2,,,,1,,intron variant,MODIFIER,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,nan,ENST00000641515.2,c.9+18C>T,,None,intron variant,2/10,1.5,,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351857,chrY,14840887,rs373089425,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;ASP;GENEINFO=NLGN...,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,0,1,Homozygous,NLGN4Y:22829,NLGN4Y,T|synonymous variant|LOW|NLGN4Y|ENSG0000016524...,T|synonymous variant|LOW|NLGN4Y|ENSG0000016524...,,,,,ENST00000382868.5:c.2247C>T,ENSP00000372320.1:p.Ala749%3D,,protein coding,8/8,,749,A,gcC/gcT,1,,synonymous variant,LOW,,,,,,,,,,,,,,,,,,

In [30]:
merged_3.Alterations.value_counts()

No    351862
Name: Alterations, dtype: int64

In [31]:
merged_3.columns

Index(['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT',
       'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF',
       'ADR', 'HET', 'HOM', 'Zygosity', 'Gene_Name', 'Gene Name', 'CSQ', 'csq',
       'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSC',
       'HGVSP', 'PolyPhen', 'BIOTYPE', 'EXON', 'INTRON', 'Protein_position',
       'Amino_acids', 'Codons', 'STRAND', 'PUBMED', 'Consequence', 'IMPACT',
       'SIFT', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF',
       'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF',
       'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF',
       'gnomADe_SAS_AF', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF',
       'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF',
       'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF',
       'MAX_AF', 'MAX_AF_POPS', 'Protein Position and Amino Acid', 'HGVSc',
  

In [32]:
merged_4 = merged_3[['Gene Name', 'Matched_Gene', 'Gene Match', 'Alterations', 'rsID', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence','Consequence_score', 'IMPACT', 'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG',
       'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp',
       'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ',
       'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE',
       'EXON', 'INTRON', 'Protein Position and Amino Acid', 'Codons', 'STRAND',
       'PUBMED']]
merged_4

,Gene Name,Matched_Gene,Gene Match,Alterations,rsID,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,IMPACT,IMPACT_score,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED
0,,,No,No,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,,,,,,None,,None,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,transcribed unprocessed pseudogene,,,nan,,1,
1,,,No,No,rs1473003496,chr1,65529,C,T,Heterozygous,5 prime UTR variant,3/10,MODIFIER,1.5,,,,,ENST00000641515.2,c.-36C>T,,None,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,protein coding,2/3,,nan,,1,
2,,,No,No,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,,,,,,None,,None,0/1,53,58,58,42,16,27.59%,4.6015E-6,36,6,12,4,,,,,,,,,,,,,,,,,,0.03299,0.02406,0,0.01429,0.5,0,0,0,0.1071,0,0,0.5,gnomADg ASJ,lncRNA,,,nan,,1,
3,,,No,No,rs1416222198,chr1,65591,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,,,,,,None,,None,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,transcribed unprocessed pseudogene,,,nan,,1,
4,,,No,No,rs1416222198,chr1,65591,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,,,,,ENST00000641515.2,c.9+18C>T,,None,0/1,42,62,62,49,13,20.97%,6.0557E-5,38,11,9,4,,,,,,,,,,,,,,,,,,0.03416,0.02306,0.1667,0.02711,0.03333,0.001639,0,0.05,0.09167,0.05357,0.02083,0.1667,gnomADg AMI,protein coding,,2/2,nan,,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351857,NLGN4Y,,No,No,rs373089425,chrY,14840887,C,T,Homozygous,synonymous variant,3/10,LOW,2.5,,,,,ENST00000382868.5,c.2247C>T,ENSP00000372320.1,p.Ala749%3D,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,protein coding,8/8,,A749,gcC/gcT,1,
351858,NLGN4Y,,No,No,rs373089425,chrY,14840887,C,T,Homozygous,synonymous variant,3/10,LOW,2.5,,,,,ENST00000382872.5,c.1572C>T,ENSP00000372325.1,p.Ala524%3D,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,protein coding,6/6,,A524,gcC/gcT,1,
351859,NLGN4Y,,No,No,rs373089425,chrY,14840887,C,T,Homozygous,non coding transcript exon variant,2/10,MODIFIER,1.5,,,,,ENST00000476359.1,n.2621C>T,,None,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,protein coding CDS not defined,9/9,,nan,,1,
351860,NLGN4Y,,No,No,rs373089425,chrY,14840887,C,T,Homozygous,synonymous variant,3/10,LOW,2.5,,,,,ENST00000643089.1,c.2136C>T,ENSP00000496594.1,p.Ala712%3D,1/1,52,10,10,0,10,100%,5.4125E-6,0,0,0,10,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,protein coding,7/7,,A712,gcC/gcT,1,


In [33]:
merged_4.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHAIGPRX1164_depth_vcf_processed.xlsx', index=False)